In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import plotly.express as px
pd.options.plotting.backend = "plotly"

In [ ]:
ds = xr.open_dataset("./data/wind_europe_2015_2019.nc")

In [ ]:
wind_speed = np.sqrt(ds['u100']**2 + ds['v100']**2)

In [ ]:
farms = pd.read_csv('./farms.csv', index_col="name")
farm_wind_ds = xr.Dataset()
for farm_name in farms.index:
    a = wind_speed.sel(latitude=farms.loc[farm_name, "latitude"], longitude=farms.loc[farm_name, "longitude"], method="nearest")
    farm_wind_ds[farm_name] = a.drop(['valid_time', 'latitude', 'longitude', 'number', 'step', 'surface'])
    
farm_load_factor_ds = xr.Dataset()
for farm_name in farms.index:
    wind = farm_wind_ds[farm_name]
    power_in = 0.5 * 1.247 * farms.loc[farm_name, "turbine_swept_area"] * wind**3 / 10**6
    power_min = 0.5 * 1.247 * farms.loc[farm_name, "turbine_swept_area"] * farms.loc[farm_name, "turbine_cut_in"]**3 / 10**6
    farm_load_factor_ds[farm_name] = xr.where(np.logical_and(wind > farms.loc[farm_name, "turbine_cut_in"], wind < farms.loc[farm_name, "turbine_cut_off"]),
                                              np.minimum(1, 16/27 * (power_in - power_min) / farms.loc[farm_name, "turbine_power"]), 0)

In [ ]:
df = farm_load_factor_ds.to_dataframe()
b= pd.DataFrame()
for farm_name in farms.index:
    b[farm_name] = df[farm_name].sort_values().reset_index(drop=True)
b['index'] = b.index / len(a)
data = pd.melt(b, id_vars = ['index'], value_vars=list(farms.index), var_name="farm", value_name="load_factor")
fig = px.line(data, x="index", y="load_factor", color='farm')
fig.show()

In [ ]:
df = farm_wind_ds.to_dataframe()
b= pd.DataFrame()
for farm_name in farms.index:
    b[farm_name] = df[farm_name].sort_values().reset_index(drop=True)
b['index'] = b.index / len(a)
data = pd.melt(b, id_vars = ['index'], value_vars=list(farms.index), var_name="farm", value_name="wind_speed")
fig = px.line(data, x="index", y="wind_speed", color='farm')
fig.show()
fig = px.box(data, x="farm", y="wind_speed")
fig.show()

In [ ]:
{
    farm_name: farm_load_factor_ds[farm_name].mean()
    for farm_name in farms.index
}

In [ ]:
df = farm_wind_ds.to_dataframe()
fig = px.scatter_matrix(df, width=1200, height=1200)
fig.update_traces(marker=dict(size=1))
fig.show()

In [ ]:
corr = farm_wind_ds.to_dataframe().corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
# impact of specific area